# Méthode semi-Lagrangienne généralisée

Cette méthode ne fait pas d'interpolation à chaque itération en temps, mais revient à la condition initiale sur la 
grille grâce aux caractéristiques.

Une amélioration possible est de procéder à une interpolation tous les $T$ itérations afin de ne pas avoir de soucis
dans la quadrature lorsque les caractéristiques tendent à converger vers une zone restreinte du domaine d'étude.

In [ ]:
using LinearAlgebra, QuadGK, Roots, FFTW
using VlasovSolvers
using Plots

import VlasovSolvers: advection!
import VlasovSolvers: samples, Particles, PIC_step!, compute_rho
using ProgressMeter

using BenchmarkTools
using LazyArrays

using LaTeXStrings
using QuadGK

In [ ]:
struct RectangleRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Vector{Float64}
    weights :: Vector{Float64}
    step :: Float64

    function RectangleRule(len, start, stop)
        points = LinRange(start, stop, len+1)[1:end-1]
        s = step(points) 
        weights = [s for _ = 1:len]
        new(len, start, stop, vec(points), weights, s)
    end
end

In [ ]:
struct TrapezoidalRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Vector{Float64}
    weights :: Vector{Float64}
    step :: Float64

    function TrapezoidalRule(len, start, stop)
        points = LinRange(start, stop, len)[1:end]
        s = step(points) 
        weights = [s for _ = 1:len]
        weights[1] /= 2
        weights[end] /= 2
        new(len, start, stop, vec(points), weights, s)
    end
end

In [ ]:
struct GaussRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Vector{Float64}
    weights :: Vector{Float64}

    function GaussRule(len, start, stop)
        points, weights = gauss(len)
        points .+= 1
        points .*= (stop - start) / 2
        points .+= start
        weights .*= (stop - start) / 2
        new(len, start, stop, points, weights)
    end
end

In [ ]:
struct KronrodRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Vector{Float64}
    weights :: Vector{Float64}
    step :: Float64

    function KronrodRule(len, start, stop)
        pts, w, _ = kronrod(len)
        weights = []
        points = []
        for i = 1:len
            push!(points, pts[i])
            push!(weights, w[i])
            push!(points, -pts[i])
            push!(weights, w[i])
        end
        push!(points, pts[end])
        push!(weights, w[end])
        
        points .+= 1
        points .*= (stop - start) / 2
        points .+= start
        weights .*= (stop - start) / 2
        len = 2*len + 1
        new(len, start, stop, points, weights)
end
    end

In [ ]:
struct rkn_order4
    a :: Array{Float64, 2}
    b̄ :: Vector{Float64}
    c :: Vector{Float64}
    b :: Vector{Float64}
    dt :: Float64
    fg ::       Array{Float64, 3}
    G ::        Array{Float64, 2}
    Vdt ::      Array{Float64, 2}
    tmpcos ::   Array{Float64, 2}
    tmpsin ::   Array{Float64, 2}
    C :: Vector{Float64}
    S :: Vector{Float64}

    function rkn_order4(X, dt)
        # a, b̄, c, b correspond to the Butcher tableau
        a = [0.0        0.0       0.0; 
            (2-√3)/12   0.0           0.0; 
            0.0         √(3)/6      0.0]
        b̄ = [(5 - 3*√3)/24,     (3+√3)/12,  (1+√3)/24]
        c = [(3+√3)/6,          (3-√3)/6,   (3+√3)/6]
        b = [(3-2*√3)/12,       1/2,        (3+2*√3)/12]

        new(a .* dt^2, b̄ .* dt^2, c .* dt, b .* dt, dt,
            zeros(Float64, 3, size(X)...), 
            similar(X), 
            similar(X), 
            similar(X),
            similar(X), [0.0], [0.0])
    end
end

In [ ]:
#= ==================== symplectic_RKN_order4!
Advect (X, V) on a time step dt using symplectic Runge-Kutta-Nystrom method of order4 [FQ10, sect.7.3, p.327, scheme1].

The equation satisfied by X is d^2(X(t)) / dt^2 = C(t)\cos(X(t)) - S(t)sin(X(t)).

RKN method considers Ẋ = V as a variable, and updates both X and V.

Using a change of variable and the jacobian of the flow being equal to 1, it is enough to know the function f on some
initial fixed grid, independant of time, to compute C(t) and S(t).

Args:
- X: matrix of positions at time t_n
- V: matrix of velocities at time t_n
- F: values of initial condition at time t_0
- dt: time step
- kx: 2π/L
- dxdv: dx*dv, where dx is the step in x and dv is the step in v

Updates X, V in place, and returns coefficients C, S at current time. 
==================== =#
function symplectic_RKN_order4!(X, V, F, rkn, kx)  
    @views begin
        for s=1:3
            rkn.G .= X .+ V .* rkn.c[s] .+ rkn.a[s, 1] .* rkn.fg[1, :, :] .+ rkn.a[s, 2] .* rkn.fg[2, :, :] .+ rkn.a[s, 3] .* rkn.fg[3, :, :]
            
            rkn.G .*= kx
     
            rkn.tmpcos .= cos.(rkn.G)
            rkn.tmpsin .= sin.(rkn.G)
            sum!(rkn.C, rkn.tmpcos .* F)
            sum!(rkn.S, rkn.tmpsin .* F)
            rkn.C ./= π
            rkn.S ./= π
            rkn.fg[s, :, :] .= (rkn.C[1] .* rkn.tmpsin .- rkn.S[1] .* rkn.tmpcos)
        end
    
        X .+= rkn.dt .* V .+ rkn.b̄[1] .* rkn.fg[1, :, :] .+ rkn.b̄[2]  .* rkn.fg[2, :, :] .+ rkn.b̄[3]  .* rkn.fg[3, :, :]
        V .+= rkn.b[1] .* rkn.fg[1, :, :] .+ rkn.b[2] .* rkn.fg[2, :, :] .+ rkn.b[3] .* rkn.fg[3, :, :]
        rkn.tmpcos .= cos.(X .* kx)
        rkn.tmpsin .= sin.(X .* kx)
        sum!(rkn.C, rkn.tmpcos .* F)
        sum!(rkn.S, rkn.tmpsin .* F)
    end
    return @views rkn.C[1], rkn.S[1]
end

In [ ]:
function strang_splitting!(X, V, F, kx, dt)  
    X .+= V .* dt/2
    C = sum(cos.(X .* kx) .* F)
    S = sum(sin.(X .* kx) .* F)
    dphidx = (-C.*sin.(X .* kx) + S.*cos.(X .* kx)) ./ π
    V .-= dt .* dphidx
    X .+= V .* dt/2
    return C, S
end

## SL Généralisée

In [ ]:
function generalized_SL(nsteps::Int64, dt::Float64, quadrulex, quadrulev, f0::Function; plotting=false::Bool)

    C = 0.0
    S = 0.0

    newX = repeat(quadrulex.points, 1, quadrulev.len)   # newX[i, j] is position at current time when starting from x0_i, v0_j
    newV = repeat(quadrulev.points', quadrulex.len, 1)  # newV[i, j] is velocity at current time when starting from x0_i, v0_j
    
    E2_elec  = Array{Float64}(undef, nsteps)
    E2_tot   = Array{Float64}(undef, nsteps)

    rkn = rkn_order4(newX, dt)
    
    progression = ProgressMeter.Progress(nstep,desc="Loop in time: ", showspeed=true)
        
    f0vals = f0.(quadrulex.points, quadrulev.points') .* quadrulex.weights .* quadrulev.weights'

    animation = @animate for istep=1:nsteps
        C, S = symplectic_RKN_order4!(newX, newV, f0vals, rkn, kx)
        # C, S = strang_splitting!(newX, newV, f0vals, kx, dt)

        E2_elec[istep] = (C^2 + S^2) / (kx*π)
        E2_tot[istep] = (E2_elec[istep] + sum(newV.^2 .* f0vals)) / 2 
        
        newX[findall(x -> x > quadrulex.stop,  newX)] .-= quadrulex.stop - quadrulex.start
        newX[findall(x -> x < quadrulex.start, newX)] .+= quadrulex.stop - quadrulex.start
 
        if plotting
            # histogram2d(vec(newX), vec(newV), weights=vec(f0vals), bins=(nx, nv).*3, normalize=true, xlabel="position", ylabel="velocity", fillcolor=:roma)
            scale = 25
            plot(vec(newX), vec(newV), vec(f0vals), seriestype=:scatter, markersize=sqrt(widthx * widthv * scale^2  / (nx*nv) / π), camera=(0, 90),markerstrokecolor="white", markerstrokewidth=0, label="", zcolor=vec(f0vals), c=:rainbow,aspect_ratio=:equal, size=(widthx, widthv).*scale)
            title!("Progression: $(round(Int64,100*progression.counter / progression.n))%")
        end
        ProgressMeter.next!(progression)
     end when plotting

     return sqrt.(E2_elec), sqrt.(E2_tot), animation, newX, newV, f0vals
end

## SL classique

In [ ]:
"""
    hmf_poisson!(fᵗ    :: Array{Complex{Float64},2},
                 mesh1 :: OneDGrid,
                 mesh2 :: OneDGrid,
                 ex    :: Array{Float64})

    Compute the electric hamiltonian mean field from the
    transposed distribution function

"""
function hmf_poisson!(fᵗ::Array{Complex{Float64},2},
        mesh1::OneDGrid,
        mesh2::OneDGrid,
        ex::Array{Float64})

    n1 = mesh1.len
    rho = mesh2.step .* vec(sum(fᵗ, dims=1)) # ≈ ∫ f(t,x_i,v)dv, i=1, ..., n1
    rho .-= sum(rho) * mesh1.step / mesh1.stop
    kernel = zeros(Float64, n1)
    k = -(mesh1.stop - mesh1.start) / (2π)
    kernel[2]   =  k    # fourier mode  1
    kernel[end] = -k    # fourier mode -1
    ex .= real(ifft(fft(rho) .* 1im .* kernel))
end


function solve_SL!(nsteps, dt, f, mesh1, mesh2; plotting=false::Bool)
    n1, n2 = size(f)
    fᵗ = zeros(Complex{Float64}, (n2,n1))
    transpose!(fᵗ, f)

    energy = Array{Float64}(undef, nsteps)

    ex = zeros(Float64, n1)
    hmf_poisson!(fᵗ, mesh1, mesh2, ex)
    advection!(fᵗ, mesh2, ex, 0.5dt)

    progression = ProgressMeter.Progress(nsteps,desc="Loop in time: ", showspeed=true)
    
    animation = @animate for istep = 1:nsteps
        energy[istep] = sqrt(sum(ex.^2) * mesh1.step)
        # etot[istep] = sqrt(energy[istep]^2 + sum()*mesh1.step*mesh2.step)
    
        if plotting
            plot(mesh1.points, mesh2.points, real(f)', size=(500, 500), st=:surface, camera=(0, 90))
            title!("Progression: $(round(Int64,100*progression.counter / progression.n))%")
        end
        advection!(f, mesh1, mesh2.points, dt)
        transpose!(fᵗ, f)
        hmf_poisson!(fᵗ, mesh1, mesh2, ex)
        advection!(fᵗ, mesh2, ex, dt)
        transpose!(f, fᵗ) 
        ProgressMeter.next!(progression)
    end when plotting
    if plotting
        return energy, ex, animation
    else
        return energy, ex, nothing
    end
end

# INPUTS

## Landau damping

In [ ]:
kx = 0.5
L = 2π / kx
ϵ = 0.001
μ = 0.0
β = 1.0
f(x,v) = (1 + ϵ * cos(kx*x)) * exp(-β * (v-μ)^2 / 2) / √(2π/β)
castest = "Landau damping\n(kx, ϵ, μ, β) = $((kx, ϵ, μ, β)))"

## Two-Stream Instability

### Première version

In [ ]:
kx = 0.2
L = 2π / kx
ϵ = 0.001
β₁ = 1
β₂ = 1
v0 = 2.4
μ₁ =  v0
μ₂ = -v0
f(x,v) = (1 + ϵ * cos(kx * x)) * (exp(-β₁*(v-μ₁)^2 / 2) + exp(-β₁*(v-μ₂)^2 / 2)) / (√(2π)) * 0.5
castest = "TSI\n(kx, β₁, β₂, μ₁, μ₂) = $((kx, β₁, β₂, μ₁, μ₂))"

### Seconde version

In [ ]:
kx = 0.2
L = 2π / kx
ϵ = 0.001
β = 1
μ =  0
f(x,v) = (1 + ϵ * cos(kx * x)) * v^2 * exp(-β*(v-μ)^2 / 2) / (√(2π))
castest = "TSI\n(kx, β, μ) = $((kx, β, μ))"

## Bump-on-Tail

In [ ]:
kx = 0.3
L = 2π / kx
ϵ = 0.04
β₁ = 1
β₂ = 1 / (0.5^2)
μ₁ = 0
μ₂ = 4.5
nb = 0.2
np = 0.9
f(x,v) = (1 + ϵ * cos(kx*x)) * (np*exp(-β₁*(v-μ₁)^2 /2) + nb*exp(-β₂*(v-μ₂)^2 / 2)) / √(2π)
castest = "Bump on tail\n(kx, ϵ, β₁, β₂, μ₁, μ₂, nb, np) = $((kx, ϵ, β₁, β₂, μ₁, μ₂, nb, np))"

## Strong Landau Damping

In [ ]:
kx = 0.5
L = 2π / kx
ϵ = 0.5
β = 1.0
μ = 0.0
f(x,v) = (1 + ϵ * cos(kx*x)) * exp(-β * v^2 / 2) / √(2π/β)
castest = "Strong Landau damping\n(kx, ϵ, μ, β) = $((kx, ϵ, μ, β)))"

# Inputs indépendants du cas test

In [ ]:
dev = CPU()

nstep = 100
dt = 1.0

In [ ]:
nx = 16
nv = 32

meshx = OneDGrid(dev, nx, 0, L);
meshv = OneDGrid(dev, nv, -10, 10);

quadX = TrapezoidalRule
quadV = TrapezoidalRule

quadrulex = quadX(nx, 0, L);
quadrulev = quadV(nv, -10, 10);

# Simulations

In [ ]:
gsl  = zeros(Complex{Float64}, (nx,nv))
@. gsl = f.(meshx.points, meshv.points');
@time E_elecsl, _, animationsl = solve_SL!(nstep, dt, gsl, meshx, meshv, plotting=false);

In [ ]:
p = false
@time E_elec, E_tot, animation, X, V, F = generalized_SL(nstep, dt, quadrulex, quadrulev, f, plotting=p);

In [ ]:
# meshx_2 = OneDGrid(dev, 64, 0, L)
# meshv_2 = OneDGrid(dev, 65, -10, 10)

meshx_2 = meshx
meshv_2 = meshv

gsl_2  = zeros(Complex{Float64}, (meshx_2.len, meshv_2.len))
@. gsl_2 = f.(meshx_2.points, meshv_2.points');
@time E_elecsl_2, _, animationsl = solve_SL!(1000, 0.05, gsl_2, meshx_2, meshv_2, plotting=false);

In [ ]:
# quadrulex_2 = quadX(64, 0, L);
# quadrulev_2 = quadV(65, -10, 10);

quadrulex_2 = quadrulex
quadrulev_2 = quadrulev

@time E_elec_2, E_tot_2, animation_2, X_2, V_2, F_2 = generalized_SL(1000, 0.05, quadrulex_2, quadrulev_2, f, plotting=p);
# nstep=1000: 3.852896 seconds (558.21 k allocations: 4.159 GiB, 9.63% gc time)
# nstep=5000: 20.110145 seconds (1.76 M allocations: 20.773 GiB, 8.93% gc time)

In [ ]:
if p
    gif(animation, "generalized_semilag.gif")
end

In [ ]:
t = (1:nstep).*dt
t_2 = (1:1000).*0.05

# energies
# plot(t, E_elec.^2, label=L"E_{elec}^2", legend=:outertopright, minorgrid=true)
# plot!(t, E_tot, label=L"E_{tot}", scaley=:log10)

# log(Energies)
plot(t, log.(E_elec),       label=L"\log(E_{elec}),\quad dt="*"$(dt)", legend=:bottomright, minorgrid=true)
plot!(t_2, log.(E_elec_2),       label=L"\log(E_{elec}), \quad dt=0.05")
plot!(t, log.(E_elecsl),    label=L"\log(E_{elec, SL}),\quad dt="*"$(dt)", ls=:dash, lw=1)
plot!(t_2, log.(E_elecsl_2),    label=L"\log(E_{elec, SL}), \quad dt=0.05", ls=:dash, lw=1)
plot!(t, log.(E_tot),       label=L"\log(E_{tot}),\quad dt="*"$(dt)")
plot!(t_2, log.(E_tot_2),       label=L"\log(E_{tot}), \quad dt=0.05")


# ============== #

# Landau damping (kx=0.5):
# plot!(x->-0.1533x - 5.6, label="Damping attendu (-0.1533)")
# E_th = abs.(4ϵ * 0.3677 .* exp.(−0.1533 .* t) .* cos.(1.4156.*t .−0.5326245) .* sqrt(L/2))
# plot!(t, log.(E_th),label="Energie theorique", ls=:dash)
# Landau damping (kx=0.4):
# plot!(x->-0.0661x - 5.3, label="Damping attendu (-0.0661)")
# E_th = 0.002.*0.42466.*abs.(cos.(1.285.*t .-0.33577)).*exp.(-0.0661.*t) # expression du bouquin, pas correcte
# E_th = abs.(4*ϵ*0.424666*exp.(-0.0661 .* t) .* cos.(1.2850 .* t .- 0.3357725) * sqrt(L/2)) # issue des calculs du bouquin en calculant correctement √(∫sin(0.5x)^2dx)
# plot!(t, log.(E_th) .- log(2),label="Energie theorique", ls=:dash)

# TSI (k,v0) = (0.2, 1.3):
# plot!(t, -0.001t .- 4.2, label=L"y=0.001t - 5.0")
# TSI (k,v0) = (0.2, 2.4):
# plot!(t, 0.2258t .- 6.4, label=L"y=0.2258t - 8.4")
# TSI (k,v0) = (0.2, 3):
# plot!(t, 0.2845t .- 6.1, label=L"y=0.2845t - 8.2")

# Strong Landau damping
# plot!(t, -0.285473t .+ 1, label=L"y=-0.285473t + 1")
# plot!(t, 0.086671t .- 3.5, label=L"y=0.086671t - 3.5")

title!(castest * "\n($(quadX)($(nx)), $(quadV)($(nv)))")
xlabel!("t (dt=$(dt))")

In [ ]:
plot(t, abs.((E_tot .- E_tot[1])) ./ E_tot[1], label=L"\frac{|E_{tot}(t) - E_{tot}(t=0)|}{E_{tot}(t=0)},\quad dt="*"$(dt)", minorgrid=true)
plot!(t_2, abs.((E_tot_2 .- E_tot_2[1])) ./ E_tot_2[1], label=L"\frac{|E_{tot}(t) - E_{tot}(t=0)|}{E_{tot}(t=0)}, \quad dt=0.05")

In [ ]:
plot(t_2, abs.((E_tot_2 .- E_tot_2[1])) ./ E_tot_2[1], label=L"\frac{|E_{tot}(t) - E_{tot}(t=0)|}{E_{tot}(t=0)}, \quad dt=0.05", minorgrid=true)